In [1]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!apt-get install -y coinor-cbc

--2025-04-23 02:36:32--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7171 (7.0K) [text/plain]
Saving to: ‘helper.py.2’

helper.py.2         100%[===================>]   7.00K  --.-KB/s    in 0s      

2025-04-23 02:36:33 (44.5 MB/s) - ‘helper.py.2’ saved [7171/7171]

Installing glpk via apt-get...
IDAES found! No need to install.
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfuly installed
cbc was successfuly installed
clp was successfuly installed
bonmin was successfuly installed
couenne was successfuly installed
ipopt_l1 was successfuly installed
 


In [3]:
import pandas as pd

df_clients = pd.read_csv('/content/clients.csv')
df_depots = pd.read_csv('/content/depots.csv')
df_vehicles = pd.read_csv('/content/vehicles.csv')

In [13]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [5]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C


print("model.D =", list(model.D.data()))
print("model.C =", list(model.C.data()))
print("model.V =", list(model.V.data()))
print("model.N =", list(model.N.data()))
print((lugares))

model.D = ['CD1']
model.C = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
model.V = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8']
model.N = ['CD1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
[(-74.153536, 4.743359), (-74.09893796560621, 4.59795431125545), (-74.07557103763986, 4.687820646838871), (-74.10708524062704, 4.70949446000624), (-74.09727965657427, 4.605029068682624), (-74.16464148202755, 4.648463876533332), (-74.12083799988112, 4.662137416953968), (-74.02213076607309, 4.697499030379109), (-74.17207549744595, 4.649416884236942), (-74.15615257246444, 4.606310650273935), (-74.09041145358674, 4.557379705282216), (-74.17802255204528, 4.591594072172954), (-74.1015410917749, 4.7564172406324055), (-74.09690889182339, 4.646217006050524), (-74.1219200708342, 

In [6]:
# Datos
precio_km = 3032.1

nodes = list(model.N)
print(nodes)

# Construcción de la URL
coordenadas = ";".join([f"{lon},{lat}" for lon, lat in lugares])
url = f"http://router.project-osrm.org/table/v1/driving/{coordenadas}?annotations=distance"

# Realiza la solicitud
respuesta = requests.get(url)

# Diccionarios para almacenar los resultados
cost = {}
distance = {}

# Verifica que la solicitud fue exitosa
if respuesta.status_code == 200:
    datos = respuesta.json()
    matriz_distancias = datos["distances"]

    for i,i1 in zip(range(len(lugares)), nodes):
        for j,j1 in zip(range(len(lugares)), nodes):
            if i != j:
                distancia_km = matriz_distancias[i][j] / 1000  # convertir a kilómetros
                precio = round(distancia_km * precio_km, 3)
            else:
                distancia_km = 9999999
                precio =9999999
            distance[(i1, j1)] = distancia_km
            cost[(i1, j1)] = precio

print(cost)
print(len(distance))


['CD1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
{('CD1', 'CD1'): 9999999, ('CD1', 'C1'): 82304.232, ('CD1', 'C2'): 53598.735, ('CD1', 'C3'): 42387.545, ('CD1', 'C4'): 80808.194, ('CD1', 'C5'): 68476.037, ('CD1', 'C6'): 56283.356, ('CD1', 'C7'): 75092.685, ('CD1', 'C8'): 72556.334, ('CD1', 'C9'): 84401.232, ('CD1', 'C10'): 102035.623, ('CD1', 'C11'): 94084.244, ('CD1', 'C12'): 48891.703, ('CD1', 'C13'): 65530.352, ('CD1', 'C14'): 31495.636, ('CD1', 'C15'): 81988.59, ('CD1', 'C16'): 102979.516, ('CD1', 'C17'): 80475.876, ('CD1', 'C18'): 60803.308, ('CD1', 'C19'): 64517.63, ('CD1', 'C20'): 40664.1, ('CD1', 'C21'): 84503.111, ('CD1', 'C22'): 73566.629, ('CD1', 'C23'): 34459.21, ('CD1', 'C24'): 93418.698, ('C1', 'CD1'): 93418.698, ('C1', 'C1'): 9999999, ('C1', 'C2'): 43226.527, ('C1', 'C3'): 58823.65, ('C1', 'C4'): 3429.002, ('C1', 'C5'): 37825.751, ('C1', 'C6'): 34616.576,

In [8]:
depot_capacity = {}
for depot in df_depots['DepotID']:
    depot_capacity[f'CD{depot}'] = 9999999

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)

{'CD1': 9999999}
{'C1': 13, 'C2': 15, 'C3': 12, 'C4': 15, 'C5': 20, 'C6': 17, 'C7': 17, 'C8': 20, 'C9': 20, 'C10': 15, 'C11': 17, 'C12': 12, 'C13': 21, 'C14': 15, 'C15': 17, 'C16': 10, 'C17': 25, 'C18': 12, 'C19': 11, 'C20': 15, 'C21': 14, 'C22': 18, 'C23': 15, 'C24': 11}
{'V1': 130, 'V2': 140, 'V3': 120, 'V4': 100, 'V5': 70, 'V6': 55, 'V7': 110, 'V8': 114}
{'V1': 170, 'V2': 200, 'V3': 180, 'V4': 90, 'V5': 100, 'V6': 170, 'V7': 150, 'V8': 140}


In [9]:
# Parámetros
model.cost = Param(model.N, model.N, initialize=lambda model, i, j: cost[(i, j)], within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda model, i, j: distance[(i, j)], within=NonNegativeReals)

# El depósito es único y fijo (por ejemplo el nodo 0)
depot = "CD1"

# Variables de decisión
model.x = Var(model.V, model.N, model.N, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# Función objetivo: minimizar el costo total de las rutas
def obj_rule(model):
    return sum(model.cost[i, j] * model.x[v, i, j]
               for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

# Cada vehículo debe salir del depósito una vez
def start_route_rule(model, v):
    return sum(model.x[v, depot, j] for j in model.N if j != depot) == 1

model.start_route = Constraint(model.V, rule=start_route_rule)

# Cada vehículo debe volver al depósito una vez
def return_route_rule(model, v):
    return sum(model.x[v, i, depot] for i in model.N if i != depot) == 1

model.return_route = Constraint(model.V, rule=return_route_rule)

# Conservación de flujo
def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

# Cada cliente debe ser visitado una sola vez por un vehículo
def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

# Capacidad del vehículo
def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)
               for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

# Rango máximo del vehículo
def vehicle_range_rule(model, v):
    return sum(model.distance[i, j] * model.x[v, i, j]
               for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)

# Reglas MTZ para evitar subtours
def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v, i] - model.u[v, j] + vehicle_capacity[v] * model.x[v, i, j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v, i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v, i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

# Resolver
solver = SolverFactory('cbc')
solver.options["seconds"] = 300  # máximo 5 minutos
results = solver.solve(model)




In [10]:
from pyomo.environ import value

# Función para construir la ruta de un vehículo
def get_vehicle_route(v, nodos, depot):
    ruta = [depot]
    actual = depot
    visitados = set()

    while True:
        siguiente = None
        for j in nodos:
            if j != actual and value(model.x[v, actual, j]) > 0.5:
                siguiente = j
                break
        if siguiente is None or siguiente == depot:
            ruta.append(depot)
            break
        ruta.append(siguiente)
        visitados.add(siguiente)
        actual = siguiente
    return ruta


print("\nRutas óptimas:")
for v in model.V:
    ruta = get_vehicle_route(v, nodes, depot)
    if len(ruta) > 2:  # Para no imprimir rutas vacías
        print(f"Vehículo {v}: " + " -> ".join(ruta))



Rutas óptimas:
Vehículo V1: CD1 -> C23 -> CD1
Vehículo V2: CD1 -> C3 -> C8 -> C5 -> C18 -> CD1
Vehículo V3: CD1 -> C22 -> C15 -> C1 -> C4 -> C17 -> C13 -> CD1
Vehículo V4: CD1 -> C14 -> CD1
Vehículo V5: CD1 -> C20 -> CD1
Vehículo V6: CD1 -> C6 -> CD1
Vehículo V7: CD1 -> C11 -> C9 -> C24 -> C10 -> C16 -> C19 -> CD1
Vehículo V8: CD1 -> C12 -> C21 -> C7 -> C2 -> CD1


In [11]:
rutas = {}

for v in model.V:
    ruta = []
    actual = depot  # Comienza en el depósito
    ruta.append(actual)

    visitados = set()
    while True:
        siguiente = None
        for j in model.N:
            if j != actual and value(model.x[v, actual, j]) > 0.5:
                siguiente = j
                break
        if siguiente is None or siguiente == depot:
            break
        ruta.append(siguiente)
        visitados.add(siguiente)
        actual = siguiente

    ruta.append(depot)  # Termina en el depósito
    if len(ruta) > 2:  # Solo agregamos rutas no vacías
        rutas[f"V{v}"] = ruta


In [16]:
import requests
import folium

# Coordenadas de cada lugar (CD1, C1, C2, ..., C24)
l = {}

for i,j in zip(nodes,lugares):
    l[i] = j
print(l)
# Crear el mapa centrado en el depósito
mapa = folium.Map(location=[4.7110, -74.0721], zoom_start=13)

# Paleta de colores para distinguir rutas por vehículo
colores = [
    "blue", "red", "green", "purple", "orange", "darkred",
    "lightblue", "cadetblue", "darkgreen", "black"
]

# Dibujar rutas por vehículo
for idx, (vehiculo, puntos) in enumerate(rutas.items()):
    ruta_coords = []

    for i in range(len(puntos) - 1):
        origen = l[puntos[i]]
        destino = l[puntos[i + 1]]

        # Consulta a OSRM
        url = f"http://router.project-osrm.org/route/v1/driving/{origen[0]},{origen[1]};{destino[0]},{destino[1]}?overview=full&geometries=geojson"
        response = requests.get(url)
        data = response.json()

        if "routes" in data and len(data["routes"]) > 0:
            ruta = data["routes"][0]["geometry"]["coordinates"]
            ruta_folium = [(lat, lon) for lon, lat in ruta]
            ruta_coords.extend(ruta_folium)

    # Dibujar la línea completa de la ruta
    folium.PolyLine(
        ruta_coords,
        color=colores[idx % len(colores)],
        weight=4,
        opacity=0.8,
        popup=vehiculo
    ).add_to(mapa)

# Agregar marcadores de todos los nodos
for nombre, coord in l.items():
    folium.Marker(
        [coord[1], coord[0]],
        popup=nombre,
        icon=folium.Icon(color="orange" if "C" in nombre else "purple")
    ).add_to(mapa)

# Mostrar el mapa
mapa


{'CD1': (-74.153536, 4.743359), 'C1': (-74.09893796560621, 4.59795431125545), 'C2': (-74.07557103763986, 4.687820646838871), 'C3': (-74.10708524062704, 4.70949446000624), 'C4': (-74.09727965657427, 4.605029068682624), 'C5': (-74.16464148202755, 4.648463876533332), 'C6': (-74.12083799988112, 4.662137416953968), 'C7': (-74.02213076607309, 4.697499030379109), 'C8': (-74.17207549744595, 4.649416884236942), 'C9': (-74.15615257246444, 4.606310650273935), 'C10': (-74.09041145358674, 4.557379705282216), 'C11': (-74.17802255204528, 4.591594072172954), 'C12': (-74.1015410917749, 4.7564172406324055), 'C13': (-74.09690889182339, 4.646217006050524), 'C14': (-74.1219200708342, 4.725912125314368), 'C15': (-74.0942948461378, 4.604168478560718), 'C16': (-74.11138839002187, 4.557320898243896), 'C17': (-74.12463941285208, 4.615869066082658), 'C18': (-74.12456164551857, 4.656402930181292), 'C19': (-74.04990580408855, 4.706188309535041), 'C20': (-74.121866801394, 4.746624762561149), 'C21': (-74.02454895778